In [1]:
import cv2
import os
import csv
import numpy as np
import shutil
from skimage.feature import hog
from skimage.morphology import skeletonize
from skimage import exposure

import numpy as np
import tensorflow as tf
from keras import layers, models, Model, Input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Folder paths
annotation_csv = 'annotations.csv'
dataset_folder = 'dataset'  # Folder containing original images

# Alpha
# alpha = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Load annotations
with open(annotation_csv, 'r') as file:
    reader = csv.reader(file)
    annotations = list(reader)[1:]  # Skip header


In [3]:
flag = annotations[0][0]
c = 0
image_list1, image_list2, image_list3, image_list4, image_list5 = [], [], [], [], []
label_list = []

# Feature extraction steps
for annotation in annotations:
    c += 1
    image_name, letter, center_x, center_y, dist_x, dist_y = annotation

    # Load original image
    # image_path = os.path.join(dataset_folder, image_name)
    img = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    image_name = image_name[image_name.find('\\')+1:image_name.find('.')]
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Error: {image_name}_{c} not found!")
        continue

    # Crop the letter using the annotation box
    start_x = int(float(center_x) - float(dist_x) / 2)
    start_y = int(float(center_y) - float(dist_y) / 2)
    end_x = int(float(center_x) + float(dist_x) / 2)
    end_y = int(float(center_y) + float(dist_y) / 2)
    cropped = img[start_y:end_y, start_x:end_x]

    # 1. Image Enhancement & Normalization (Histogram Equalization)
    resized = cv2.resize(cropped, (64, 64))
    enhanced = cv2.equalizeHist(resized)
    norm_image = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    image_list1.append(norm_image)
    label_list.append(letter)

    # 2. Segmentation (Thresholding)
    _, segmented = cv2.threshold(norm_image, 30, 255, cv2.THRESH_BINARY_INV)
    image_list2.append(segmented)

    # 3. Edge Detection (Canny)
    edges = cv2.Canny(segmented, 100, 200)
    image_list3.append(edges)

    # 4. Skeletonization
    binary = segmented / 255  # Convert to binary (0, 1)
    skeleton = skeletonize(binary).astype(np.uint8) * 255
    image_list4.append(skeleton)

    # 5. HOG (Histogram of Oriented Gradients)
    hog_features, hog_image = hog(segmented, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)
    image_list5.append(hog_image)


In [4]:
label_list

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'H',
 'H'

In [4]:
alphaIndexes = {}
for i in alpha:
    alphaIndexes[i] = []
for i in range(len(label_list)):
    try:
        alphaIndexes[label_list[i]].append(i)
    except:
        pass
# alphaIndexes

In [5]:
indexes = [i for i in range(len(label_list))]
xrl, xtl = train_test_split(indexes, test_size=0.2, random_state=42)

In [14]:
splitIndexes = {}
for i in alpha:
    xrl, xtl = train_test_split(alphaIndexes[i], test_size=0.2, random_state=42)
    # print(i, len(xrl), len(xtl), sep=", ")
    times = 500//len(xrl)
    print(times, end=" ")
    xrl1 = xrl.copy()
    for j in range(times): xrl1.extend(xrl)
    print(i, len(xrl1), len(xtl), sep=", ")
    splitIndexes[i] = [sorted(xrl1), sorted(xtl)]
# splitIndexes

2 a, 735, 62
6 b, 532, 20
3 c, 568, 36
2 d, 525, 44
1 e, 810, 102
11 f, 516, 11
5 g, 528, 22
3 h, 536, 34
2 i, 720, 61
50 j, 510, 3
15 k, 512, 8
4 l, 580, 29
4 m, 535, 27
1 n, 636, 80
1 o, 602, 76
5 p, 600, 25
22 q, 506, 6
2 r, 621, 52
2 s, 702, 59
1 t, 580, 73
4 u, 615, 31
10 v, 528, 12
9 w, 520, 13
22 x, 506, 6
7 y, 504, 16
33 z, 510, 4


In [21]:
xrl, xtl = [], []
for i in alpha:
    xrl.extend(splitIndexes[i][0])
    xtl.extend(splitIndexes[i][1])

In [20]:
xrl

[0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 4,
 4,
 4,
 6,
 6,
 6,
 8,
 8,
 8,
 10,
 10,
 10,
 595,
 595,
 595,
 611,
 611,
 611,
 631,
 631,
 631,
 643,
 643,
 643,
 646,
 646,
 646,
 661,
 661,
 661,
 687,
 687,
 687,
 692,
 692,
 692,
 706,
 706,
 706,
 717,
 717,
 717,
 721,
 721,
 721,
 727,
 727,
 727,
 808,
 808,
 808,
 814,
 814,
 814,
 822,
 822,
 822,
 826,
 826,
 826,
 840,
 840,
 840,
 845,
 845,
 845,
 856,
 856,
 856,
 912,
 912,
 912,
 919,
 919,
 919,
 929,
 929,
 929,
 944,
 944,
 944,
 957,
 957,
 957,
 974,
 974,
 974,
 985,
 985,
 985,
 989,
 989,
 989,
 1006,
 1006,
 1006,
 1010,
 1010,
 1010,
 1044,
 1044,
 1044,
 1048,
 1048,
 1048,
 1051,
 1051,
 1051,
 1056,
 1056,
 1056,
 1062,
 1062,
 1062,
 1145,
 1145,
 1145,
 1163,
 1163,
 1163,
 1178,
 1178,
 1178,
 1190,
 1190,
 1190,
 1193,
 1193,
 1193,
 1198,
 1198,
 1198,
 1212,
 1212,
 1212,
 1229,
 1229,
 1229,
 1238,
 1238,
 1238,
 1307,
 1307,
 1307,
 1312,
 1312,
 1312,
 1332,
 1332,
 1332,
 1357,
 1357,
 1357,
 1418,


In [22]:
def lettersSplit(imgList, labelList, uniqueLabels=alpha):
    labelToIdx = {label: idx for idx, label in enumerate(uniqueLabels)}
    numericalLabel = np.array([labelToIdx[label] for label in labelList])

    # Split data into training and testing sets
    x_train, x_test, y_train, y_test = [imgList[i] for i in xrl], [imgList[i] for i in xtl], [numericalLabel[i] for i in xrl], [numericalLabel[i] for i in xtl]

    # Normalize the image data (scale pixel values to range [0, 1])
    x_train = np.array(x_train) / 255.0
    x_test = np.array(x_test) / 255.0

    # Convert labels to categorical (one-hot encoding)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(uniqueLabels))
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(uniqueLabels))
    x_train = np.expand_dims(x_train, axis=-1)  # (num_samples, height, width, 1)
    x_test = np.expand_dims(x_test, axis=-1)

    # Set input shape based on your image data (e.g., (height, width, channels))
    input_shape = x_train.shape[1:]  # Height, Width, Channels
    return input_shape, x_train, x_test, y_train, y_test

In [23]:
# Define the CNN model
def cnnTraining(input_shape):
    # First convolutional layer
    x = layers.Conv2D(32, (3,3), activation = 'relu')(input_shape)
    x = layers.MaxPooling2D((2,2))(x)

    # Second convolutional layer
    x = layers.Conv2D(64,(3, 3), activation = 'relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Third convolutional layer
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Flatten the output
    x = layers.Flatten()(x)
    return x

In [24]:
unique_labels = sorted(list(set(label_list)))  # Get unique classes
num_classes = len(unique_labels)

input_shape1, x_train_1, x_test_1, y_train_1, y_test_1 = lettersSplit(image_list1, label_list, unique_labels)
input_shape2, x_train_2, x_test_2, y_train_2, y_test_2 = lettersSplit(image_list2, label_list, unique_labels)
input_shape3, x_train_3, x_test_3, y_train_3, y_test_3 = lettersSplit(image_list3, label_list, unique_labels)
input_shape4, x_train_4, x_test_4, y_train_4, y_test_4 = lettersSplit(image_list4, label_list, unique_labels)
input_shape5, x_train_5, x_test_5, y_train_5, y_test_5 = lettersSplit(image_list5, label_list, unique_labels)


In [25]:
input1 = Input(shape = input_shape1)
input2 = Input(shape= input_shape2)
input3 = Input(shape= input_shape3)
input4 = Input(shape= input_shape4)
input5 = Input(shape= input_shape5)

# Create the model
cnn1 = cnnTraining(input1)
cnn2 = cnnTraining(input2)
cnn3 = cnnTraining(input3)
cnn4 = cnnTraining(input4)
cnn5 = cnnTraining(input5)

concatenated = layers.Concatenate()([cnn1, cnn2, cnn3, cnn4, cnn5])

x = layers.Dense(128, activation = 'relu')(concatenated)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=[input1, input2, input3, input4, input5], outputs = output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 62, 62,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 62, 62,    │        320 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 62, 62,    │        320 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 62, 62,    │        320 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 62, 62,    │        320 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 31, 31,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 31, 31,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 31, 31,    │          0 │ conv2d_6[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 31, 31,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 31, 31,    │          0 │ conv2d_12[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 29, 29,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 29, 29,    │     18,496 │ max_pooling2d_3[

 Total params: 1,765,044 (6.73 MB)

 Trainable params: 1,765,044 (6.73 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
x_train_1.shape

(15037, 64, 64, 1)

In [27]:
history = model.fit([x_train_1, x_train_2, x_train_3, x_train_4, x_train_5], [y_train_1, y_train_2, y_train_3, y_train_4, y_train_5], epochs = 20, batch_size=32, validation_data=([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5], [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]))

test_loss, test_acc = model.evaluate([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5], [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5])
print(f"Test accuracy: {test_acc}")

Epoch 1/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 25s 48ms/step - accuracy: 0.4380 - loss: 2.0052 - val_accuracy: 0.7018 - val_loss: 1.1333
Epoch 2/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.9267 - loss: 0.2321 - val_accuracy: 0.6623 - val_loss: 1.5180
Epoch 3/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 23s 48ms/step - accuracy: 0.9761 - loss: 0.0723 - val_accuracy: 0.7039 - val_loss: 1.7611
Epoch 4/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 0.9888 - loss: 0.0343 - val_accuracy: 0.6886 - val_loss: 1.7831
Epoch 5/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 0.9895 - loss: 0.0320 - val_accuracy: 0.7007 - val_loss: 1.9862
Epoch 6/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.9902 - loss: 0.0334 - val_accuracy: 0.6864 - val_loss: 2.1783
Epoch 7/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.9924 - loss: 0.0231 - val_accuracy: 0.6996 - val_loss: 2.0683
Epoch 8/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.9931 - loss: 0.0208 - 